In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/amnam/.secret/yelp_api.json') as f:
    login = json.load(f)

yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)


In [3]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Burger'

In [4]:
JSON_FILE = "Data/results_in_progress_NY_burger.json"
JSON_FILE

'Data/results_in_progress_NY_burger.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_burger.json not found. Saving empty list to file.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results

16100

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [10]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

805

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [12]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/805 [00:00<?, ?it/s]

In [13]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8xpGj_YkI2X6QxqXUJPw7g,7th-street-burger-new-york,7th Street Burger,https://s3-media2.fl.yelpcdn.com/bphoto/GYqi52...,False,https://www.yelp.com/biz/7th-street-burger-new...,267,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 40.72697, 'longitude': -73.98546}","[pickup, delivery]",$$,"{'address1': '91 E 7th St', 'address2': None, ...",+16464906797,(646) 490-6797,2280.348538
1,H26qMBb0OFNn2xmSTM1Phw,kings-of-kobe-new-york-new-york-2,Kings of Kobe - New York,https://s3-media1.fl.yelpcdn.com/bphoto/E09YEC...,False,https://www.yelp.com/biz/kings-of-kobe-new-yor...,418,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 40.761172, 'longitude': -74.000307}","[pickup, delivery]",$$,"{'address1': '650 W 42nd St', 'address2': None...",+16463705121,(646) 370-5121,6283.378163
2,vTyC--4tl7_AzHqbFpCqPg,not-today-maybe-tomorrow-brooklyn,Not Today Maybe Tomorrow,https://s3-media2.fl.yelpcdn.com/bphoto/cUkl9d...,False,https://www.yelp.com/biz/not-today-maybe-tomor...,19,"[{'alias': 'bars', 'title': 'Bars'}]",4.5,"{'latitude': 40.714697, 'longitude': -73.944161}","[pickup, delivery]",NaN,"{'address1': '749 Metropolitan Ave', 'address2...",+13477630591,(347) 763-0591,3080.401556
3,m86CNmYThtzzWB3nPnrhbg,emily-west-village-new-york,Emily - West Village,https://s3-media2.fl.yelpcdn.com/bphoto/6zPD8M...,False,https://www.yelp.com/biz/emily-west-village-ne...,944,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.729375015649076, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '35 Downing St', 'address2': '', ...",+19179356434,(917) 935-6434,3244.581130
4,UA2M9QFZghe-9th2KwLoWQ,burger-and-lobster-flatiron-nyc-new-york,Burger & Lobster - Flatiron NYC,https://s3-media1.fl.yelpcdn.com/bphoto/2Z3Cn2...,False,https://www.yelp.com/biz/burger-and-lobster-fl...,5866,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 40.74007, 'longitude': -73.99344}","[pickup, delivery]",$$,"{'address1': '39 W 19th St', 'address2': '', '...",+16468337532,(646) 833-7532,3867.245488


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
15,BCwSqk9WY5woPkWEkvWkMg,bronsons-burgers-new-york,Bronson's Burgers,https://s3-media1.fl.yelpcdn.com/bphoto/ukIdYP...,False,https://www.yelp.com/biz/bronsons-burgers-new-...,51,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 40.72317, 'longitude': -73.99568}",[],$$,"{'address1': '250 Mulberry St', 'address2': ''...",+12128886280,(212) 888-6280,2277.702132
16,ruYQv7KVDvmWSF82sM3ZhQ,milk-burger-new-york-2,Milk Burger,https://s3-media3.fl.yelpcdn.com/bphoto/aV3_jG...,False,https://www.yelp.com/biz/milk-burger-new-york-...,13,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 40.72095, 'longitude': -73.9829}","[pickup, delivery]",NaN,"{'address1': '321 E Houston St', 'address2': '...",+12123601988,(212) 360-1988,1586.576743
17,qOp4T7UxfL6MgO41dN3Qlg,whitmans-new-york,Whitmans,https://s3-media2.fl.yelpcdn.com/bphoto/IZ47pj...,False,https://www.yelp.com/biz/whitmans-new-york?adj...,674,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 40.727942, 'longitude': -73.984469}","[pickup, delivery]",$$,"{'address1': '406 E 9th St', 'address2': '', '...",+19172612858,(917) 261-2858,2373.243836
18,uYLIo8FCr9Ug5SFORDdsPA,the-burgary-new-york,The Burgary,https://s3-media4.fl.yelpcdn.com/bphoto/X9GgkF...,False,https://www.yelp.com/biz/the-burgary-new-york?...,300,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.0,"{'latitude': 40.71944096840517, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '67 Clinton St', 'address2': '', ...",+12125296900,(212) 529-6900,1472.731131
19,nnMEtKLL0lyaXyXACRkVEQ,no-7-brooklyn-3,no. 7,https://s3-media2.fl.yelpcdn.com/bphoto/si-Vh_...,False,https://www.yelp.com/biz/no-7-brooklyn-3?adjus...,8,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 40.67831, 'longitude': -73.96843}",[],NaN,"{'address1': '627 Vanderbilt Ave', 'address2':...",,,3314.736877


In [14]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [15]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_burger.csv.gz', compression='gzip',index=False)